# MediCaT Development Notebook

In [ ]:
import json
import pandas as pd
import os

In [ ]:
medicat_directory = "../../data_links/data/medicat_release/release"
medicat_figures_directory = f"{medicat_directory}/figures"
subcaptions_info_filename = f"{medicat_directory}/subcaptions_public.jsonl"
with open(subcaptions_info_filename, "r") as f:
    subcaptions_info = [json.loads(line) for line in f]
subcaptions_df = pd.DataFrame(subcaptions_info)

In [9]:
subcaptions_df.columns

Index(['pdf_hash', 'fig_key', 'fig_uri', 'text', 'width', 'height',
       'subcaptions', 'subfigures', 'tokens'],
      dtype='object')

In [24]:
subcaptions_df["filenames"] = subcaptions_df["pdf_hash"] + "_" + subcaptions_df["fig_uri"]

In [ ]:
#list all image names in medicat_figures_directory
image_filenames = os.listdir(medicat_figures_directory)

False

In [26]:
#show figure
from PIL import Image
image = Image.open(f"{medicat_figures_directory}/{subcaptions_df['filenames'][0]}")
image.show()

I0000 00:00:1765479380.580299 1374399 voice_transcription.cc:58] Registering VoiceTranscriptionCapability
[1374360:1374402:1211/195620.668423:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT


## Randomly Select some to properly label the subfigures

In [ ]:
import os
import random
import shutil
from pathlib import Path
from tqdm.notebook import tqdm

# --- KONFIGURATION ---

# 1. Wo liegen die MedICaT Bilder? (Passe den Pfad an, falls nötig)
source_dir = Path("../../data_links/data/medicat_release/release/figures")

# 2. Wohin sollen die 500 Bilder?
target_dir = Path("../../dataset/02_assets/MedICaT_Real-compound_NOTYETLABELED")

# 3. Wie viele Bilder?
SAMPLE_SIZE = 500

# --- LOGIK ---

# Zielordner erstellen
target_dir.mkdir(parents=True, exist_ok=True)

# Überprüfen ob Quelle existiert
if not source_dir.exists():
    print(f"❌ Fehler: Quellordner nicht gefunden: {source_dir}")
else:
    # Alle Bild-Dateien auflisten
    # MedICaT Bilder sind meist .jpg oder .png
    valid_extensions = {".jpg", ".jpeg", ".png", ".pdf"}
    
    print("⏳ Scanne MedICaT Ordner (das kann kurz dauern)...")
    all_files = [
        f for f in source_dir.iterdir() 
        if f.is_file() and f.suffix.lower() in valid_extensions
    ]
    
    total_files = len(all_files)
    print(f"📂 Gefunden: {total_files} Bilder im Quellordner.")

    if total_files < SAMPLE_SIZE:
        print(f"⚠️ Warnung: Der Ordner enthält weniger als {SAMPLE_SIZE} Bilder. Kopiere alles ({total_files}).")
        selected_files = all_files
    else:
        print(f"🎲 Wähle zufällig {SAMPLE_SIZE} Bilder aus...")
        selected_files = random.sample(all_files, SAMPLE_SIZE)

    # Kopiervorgang mit Progress Bar
    count_success = 0
    
    for src_file in tqdm(selected_files, desc="Kopiere Bilder"):
        dst_file = target_dir / src_file.name
        
        try:
            # --- WÄHLE HIER DEINE METHODE ---
            
            # OPTION A: KOPIEREN (Empfohlen, sicher)
            shutil.copy2(src_file, dst_file)
            
            # OPTION B: VERSCHIEBEN (Löscht Original! Nur nutzen wenn sicher!)
            # shutil.move(str(src_file), str(dst_file))
            
            count_success += 1
            
        except Exception as e:
            print(f"⚠️ Fehler bei {src_file.name}: {e}")

    print("-" * 30)
    print(f"✅ Fertig! {count_success} Bilder liegen jetzt in:")
    print(f"📂 {target_dir.resolve()}")

⏳ Scanne MedICaT Ordner (das kann kurz dauern)...
📂 Gefunden: 217002 Bilder im Quellordner.
🎲 Wähle zufällig 500 Bilder aus...


Kopiere Bilder:   0%|          | 0/500 [00:00<?, ?it/s]

------------------------------
✅ Fertig! 500 Bilder liegen jetzt in:
📂 /home/tobiasponeschtu/Documents/11818774_CompoundFigureSeparation/dataset/to_label/MedICaT
